<a href="https://colab.research.google.com/github/NataliaKolesnik/Intergalactic_hackathon/blob/master/1_RZD_all_st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# Монтирую свой гугл диск
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Путь к файлам
path = '/content/drive/MyDrive/RZD'
# Чтоб все колонки видеть
pd.set_option('display.max_columns', None)
# Отображать данные float с 1 знаком после запятой
pd.options.display.float_format ='{:,.0f}'.format

In [4]:
# Загружаем ДатаФреймы
df = pd.read_csv(path+'/statistics-07-20.csv', sep=';')

In [5]:
# Так как в колонке оперции есть пропуски, а она для нас очень важная, то разделим датасет на две части (без пропусков и пропуски)
# Если не понадобиться, то потом датасет пропусков удалим
df_oper_nan = df[(df['operation_car'].isna()) & (df['operation_st_esr'].isna())].copy() # тут пропуски operation_car и по operation_st_esr
df_oper = df[(df['operation_car'].isna() == False) & (df['operation_st_esr'].isna() == False)].copy()


In [6]:
df_oper.shape

(3648085, 21)

In [7]:
# Проверяем где и сколько nan 
df_oper.isna().sum()

index_train         2221822
length              1714849
car_number                0
destination_esr       58255
adm                 2543576
danger              3536099
gruz                1771902
loaded              1104509
operation_car             0
operation_date            0
operation_st_esr          0
operation_st_id           0
operation_train     2221823
receiver            1764038
rodvag              1117313
rod_train           2623213
sender              1764038
ssp_station_esr     2221822
ssp_station_id      2228617
tare_weight         2543576
weight_brutto       2622868
dtype: int64

In [8]:
# Заменяем если груз не опасный 0, а опасный остается 1
df_oper['danger'] = df_oper['danger'].fillna(0) 
# Заменяем 0, чтобы обработать, но будеб подразумевать, что 0 - это неизвестно что
# Если код груза не указан, то будет 0
df_oper['gruz'] = df_oper['gruz'].fillna(0)
# Станция назначения вагона - nan пока заменим 0
df_oper['destination_esr'] = df_oper['destination_esr'].fillna(0)

# Преобразуем данные в нужные форматы
df_oper['operation_st_esr'] = df_oper['operation_st_esr'].astype('int64')
df_oper['gruz'] = df_oper['gruz'].astype('int64')
df_oper['destination_esr'] = df_oper['destination_esr'].astype('int64')

df_oper['operation_car'] = df_oper['operation_car'].astype('int64')
df_oper['operation_date'] = pd.to_datetime(df_oper['operation_date'])


In [9]:
# Удалим колонки с большим количеством пропусков
col_del = ['index_train', 'loaded', 'operation_train', 'rod_train', 'ssp_station_esr', 'ssp_station_id', 'weight_brutto', 'operation_st_id', 'length', 'adm', 'receiver', 'sender']
df_oper = df_oper.drop(col_del, axis=1)

In [10]:
df_oper

,car_number,destination_esr,danger,gruz,operation_car,operation_date,operation_st_esr,rodvag,tare_weight
0,62827035,984700,0,0,81,2020-07-16 03:40:00,984502,NaN,NaN
1,62827035,862201,0,421034,18,2020-07-16 14:10:00,984700,60,249
2,62827316,863007,0,0,80,2020-07-16 14:50:00,863007,NaN,NaN
3,62827381,862108,0,0,80,2020-07-16 14:16:00,862108,NaN,NaN
4,62845730,887904,0,421034,19,2020-07-16 15:15:00,967808,60,248
...,...,...,...,...,...,...,...,...,...
4189910,62827548,76404,0,161185,4,2020-07-16 03:29:00,860009,60,NaN
4189911,62827548,76404,0,161185,4,2020-07-16 03:30:00,860009,60,NaN
4189912,62827910,862201,0,0,3,2020-07-15 20:00:00,862201,60,NaN
4189913,62827514,968302,0,0,80,2020-07-16 02:37:00,968302,NaN,NaN


In [11]:
# Проверяем где и сколько nan 
df_oper.isna().sum()

car_number                0
destination_esr           0
danger                    0
gruz                      0
operation_car             0
operation_date            0
operation_st_esr          0
rodvag              1117313
tare_weight         2543576
dtype: int64

### Формируем датасет для анализа

In [12]:
# # Для этого сначала отсотрируем наш датасет
df_oper = df_oper.sort_values(['operation_st_esr', 'car_number', 'operation_date'], ignore_index=True)

In [13]:
# Прогоним циклом, так как пока не знаю как это сделать быстрее, потом найду
# Создадим словари станция-вагон, чтоб агрегировать данные
oper_dict = df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()
time_dict = df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()
stnazn_dict =	df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()
danger_dict = df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()	
gruz_dict = df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()
rodvag_dict = df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()	
tare_weight_dict = df_oper.groupby(['operation_st_esr', 'car_number'])['operation_car'].count().to_dict()
# Заменим значение ключей словарей на пустные списки
for key, val in oper_dict.items():
    oper_dict[key] = []
    time_dict[key] = []
    stnazn_dict[key] = []
    danger_dict[key] = []
    gruz_dict[key] = []
    rodvag_dict[key] = []
    tare_weight_dict[key] = []

In [14]:
# Прогоним циклом, так как пока не знаю как это сделать быстрее, потом найду
# Агрегируем данные для дальнейшего формирования колонок
for i in range(df_oper.shape[0]):
     st = df_oper['operation_st_esr'].iloc[i]
     vag = df_oper['car_number'].iloc[i]
     oper = df_oper['operation_car'].iloc[i]
     time = df_oper['operation_date'].iloc[i]
     stnazn = df_oper['destination_esr'].iloc[i]
     danger = df_oper['danger'].iloc[i]
     gruz = df_oper['gruz'].iloc[i]
     rodvag = df_oper['rodvag'].iloc[i]
     tare_weight = df_oper['tare_weight'].iloc[i]
     oper_dict[(st, vag)].append(oper) 
     time_dict[(st, vag)].append(time)
     stnazn_dict[(st, vag)].append(stnazn)
     danger_dict[(st, vag)].append(danger)
     gruz_dict[(st, vag)].append(gruz)
     rodvag_dict[(st, vag)].append(rodvag)
     tare_weight_dict[(st, vag)].append(tare_weight)

In [15]:
# Уберем повторяющиеся операции в процессе, такие как 4 4 4, т.е. несколько раз присоединение вагона к  поезду. 
# Похоже на ошибку, так как невозможно присоединить то, что уже присоеденено
for key, val in oper_dict.items():
    l_op = [val[0]]
    if len(val)>1:
        for i in range(1,len(val)):
            if val[i] != val[i-1]:
                l_op.append(val[i])
    oper_dict[key] = l_op
   

In [16]:
# Так как поудаляли дубли, то это уже не нужно. Но код не убираю, если повторяющиеся операции важны
# time_oper={}
# for key, val in time_dict.items():
#     l = len(val)
#     time_oper[key] = []
#     if l > 1:
#        for i in range(l-1):
#           t = (val[i+1]-val[i])
#           t = t.seconds/60 + t.days*24*60 
#           t = int(t)
#           time_oper[key].append(t)
#     else:
#        t = val[0]-val[0]
#        t = t.seconds/60 + t.days*24*60 
#        t = int(t)
#        time_oper[key].append(t)


In [18]:
st = []
vag = []
oper_start = []
oper_finish = []
oper_all = []
oper_count = []
time_start = []
time_finish = []
time_delta = []
#time_oper_all = []
stnazn = []
danger = []
gruz = []
rodvag = []
tare_weight = []
for key, _ in time_dict.items():
    st.append(key[0])
    vag.append(key[1])
    oper_start.append(oper_dict[key][0])
    oper_finish.append(oper_dict[key][-1])
    oper_all.append(str(oper_dict[key]))
    oper_count.append(len(oper_dict[key]))
    time_start.append(time_dict[key][0])
    time_finish.append(time_dict[key][-1])
    time_delta.append(time_dict[key][-1]-time_dict[key][0])
    stnazn.append(stnazn_dict[key][0])
    danger.append(danger_dict[key][0])
    gruz.append(gruz_dict[key][0])
    rodvag.append(rodvag_dict[key][0])
    tare_weight.append(tare_weight_dict[key][0])
    #time_oper_all.append(str(time_oper[key]))
    #print(st, vag, time_delta, time_)


    
df_svod_all_st = pd.DataFrame({'станция':st, 'вагон':vag, 'операция_первая':oper_start, 'операция_последняя':oper_finish, 'все_операции':oper_all,
                        'количество_операций':oper_count, 'время_первой_операции':time_start, 'время_последней_операции':time_finish, 'разница_времени':time_delta,
                        'станция_назначения':stnazn, 'опасный_груз':danger, 'код_груза':gruz, 'род_вагона':rodvag, 'вес_тары_вагона':tare_weight})

df_svod_all_st

,станция,вагон,операция_первая,операция_последняя,все_операции,количество_операций,время_первой_операции,время_последней_операции,разница_времени,станция_назначения,опасный_груз,код_груза,род_вагона,вес_тары_вагона
0,830003,24286957,3,81,"[3, 80, 28, 19, 81]",5,2020-07-19 14:49:00,2020-07-20 15:12:00,1 days 00:23:00,830003,0,0,20,NaN
1,830003,24577066,3,80,"[3, 80]",2,2020-07-16 20:33:00,2020-07-16 20:34:00,0 days 00:01:00,830003,0,0,20,NaN
2,830003,24578908,3,81,"[3, 80, 28, 19, 81]",5,2020-07-19 14:49:00,2020-07-20 15:12:00,1 days 00:23:00,830003,0,0,20,NaN
3,830003,24579856,3,4,"[3, 80, 19, 81, 4, 3, 18, 4]",8,2020-07-26 18:38:00,2020-07-29 06:02:00,2 days 11:24:00,830003,0,0,20,NaN
4,830003,24622276,3,4,"[3, 80, 28, 19, 81, 4]",6,2020-07-15 23:32:00,2020-07-17 04:27:00,1 days 04:55:00,830003,0,0,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980487,998100,63750566,3,4,"[3, 80, 21, 19, 81, 4]",6,2020-07-25 05:24:00,2020-07-29 15:00:00,4 days 09:36:00,998100,0,0,60,NaN
980488,998100,63968861,3,21,"[3, 80, 21]",3,2020-07-29 04:41:00,2020-07-30 08:20:00,1 days 03:39:00,998100,0,0,60,NaN
980489,998100,64541147,3,20,"[3, 78, 20]",3,2020-07-29 04:41:00,2020-07-30 08:30:00,1 days 03:49:00,998100,0,0,60,NaN
980490,998100,66288432,3,4,"[3, 80, 21, 19, 81, 4]",6,2020-07-25 05:24:00,2020-07-29 15:00:00,4 days 09:36:00,998100,0,0,60,NaN


In [19]:
df_svod_all_st['изм_станции'] = (df_svod_all_st['станция'] - df_svod_all_st['станция_назначения'])
df_svod_all_st['изм_станции'] = df_svod_all_st['изм_станции'].apply(lambda x: 0 if x==0 else 1) 

In [20]:
df_svod_all_st[df_svod_all_st['изм_станции'] != 0]

,станция,вагон,операция_первая,операция_последняя,все_операции,количество_операций,время_первой_операции,время_последней_операции,разница_времени,станция_назначения,опасный_груз,код_груза,род_вагона,вес_тары_вагона,изм_станции
8,830003,28036036,4,4,[4],1,2020-07-15 22:44:00,2020-07-16 01:20:00,0 days 02:36:00,835609,0,693227,20,NaN,1
11,830003,29000692,18,4,"[18, 4]",2,2020-07-16 18:30:00,2020-07-17 02:13:00,0 days 07:43:00,831400,0,421195,20,271,1
12,830003,29014966,18,4,"[18, 4]",2,2020-07-29 20:33:00,2020-07-30 09:12:00,0 days 12:39:00,831504,0,421195,20,253,1
13,830003,29014974,19,4,"[19, 81, 4]",3,2020-07-25 12:39:00,2020-07-26 09:27:00,0 days 20:48:00,831504,0,421034,20,252,1
15,830003,29029394,18,4,"[18, 4]",2,2020-07-29 20:52:00,2020-07-30 09:12:00,0 days 12:20:00,831504,0,421195,20,253,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980448,998100,53927075,4,80,"[4, 3, 80]",3,2020-07-14 12:33:00,2020-07-26 03:20:00,11 days 14:47:00,990005,0,214043,70,NaN,1
980454,998100,55095004,10,10,[10],1,2020-07-23 17:00:00,2020-07-23 17:00:00,0 days 00:00:00,960103,0,316073,60,233,1
980455,998100,55129209,11,4,"[11, 81, 4]",3,2020-07-25 05:16:00,2020-07-25 12:40:00,0 days 07:24:00,799101,0,242016,60,240,1
980457,998100,55139349,10,10,[10],1,2020-07-23 17:00:00,2020-07-23 17:00:00,0 days 00:00:00,960103,0,316073,60,244,1


In [21]:
df_svod_all_st.to_csv(path+'/df_svod_all_st.csv', index=False)

In [22]:
df_svod_all_st

,станция,вагон,операция_первая,операция_последняя,все_операции,количество_операций,время_первой_операции,время_последней_операции,разница_времени,станция_назначения,опасный_груз,код_груза,род_вагона,вес_тары_вагона,изм_станции
0,830003,24286957,3,81,"[3, 80, 28, 19, 81]",5,2020-07-19 14:49:00,2020-07-20 15:12:00,1 days 00:23:00,830003,0,0,20,NaN,0
1,830003,24577066,3,80,"[3, 80]",2,2020-07-16 20:33:00,2020-07-16 20:34:00,0 days 00:01:00,830003,0,0,20,NaN,0
2,830003,24578908,3,81,"[3, 80, 28, 19, 81]",5,2020-07-19 14:49:00,2020-07-20 15:12:00,1 days 00:23:00,830003,0,0,20,NaN,0
3,830003,24579856,3,4,"[3, 80, 19, 81, 4, 3, 18, 4]",8,2020-07-26 18:38:00,2020-07-29 06:02:00,2 days 11:24:00,830003,0,0,20,NaN,0
4,830003,24622276,3,4,"[3, 80, 28, 19, 81, 4]",6,2020-07-15 23:32:00,2020-07-17 04:27:00,1 days 04:55:00,830003,0,0,20,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980487,998100,63750566,3,4,"[3, 80, 21, 19, 81, 4]",6,2020-07-25 05:24:00,2020-07-29 15:00:00,4 days 09:36:00,998100,0,0,60,NaN,0
980488,998100,63968861,3,21,"[3, 80, 21]",3,2020-07-29 04:41:00,2020-07-30 08:20:00,1 days 03:39:00,998100,0,0,60,NaN,0
980489,998100,64541147,3,20,"[3, 78, 20]",3,2020-07-29 04:41:00,2020-07-30 08:30:00,1 days 03:49:00,998100,0,0,60,NaN,0
980490,998100,66288432,3,4,"[3, 80, 21, 19, 81, 4]",6,2020-07-25 05:24:00,2020-07-29 15:00:00,4 days 09:36:00,998100,0,0,60,NaN,0
